# TNS-DESI Matches

Access a list of transients reported to the [Transient Name Server](https://wis-tns.weizmann.ac.il/) (TNS) between Jan. 1 2020 and June 10, 2022 and match fibers to transients. DESI observations are taken from the [redshift database](https://desi.lbl.gov/trac/wiki/DESIDatabase) maintained at NERSC by Rob Knop.

TNS data were downloaded using a [Python script](https://github.com/desihub/timedomain/blob/master/tns/tns_download.py) available in the [DESI Transients/Low-z GitHub space](https://github.com/desihub/timedomain).

Note: TNS is the primary IAU database for reporting public transients. It includes alerts from all major optical transient facilities and surveys such as ZTF, PTF, ASASSN, etc.

In [1]:
from desispec.io import read_spectra
from desispec.coaddition import coadd_cameras
from desispec.interpolation import resample_flux
from desispec.resolution import Resolution

import redrock.templates

rrtemplates = dict()
for filename in redrock.templates.find_templates():
    t = redrock.templates.Template(filename)
    rrtemplates[(t.template_type, t.sub_type)] = t

from astropy import units as u
from astropy.table import Table, vstack, hstack
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.time import Time
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

from datetime import datetime, timedelta

from scipy.ndimage import gaussian_filter1d

import os
from glob import glob

import psycopg2

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

DEBUG: Read templates from /global/common/software/desi/cori/desiconda/20211217-2.0.0/code/redrock-templates/master
DEBUG: Using default redshift range -0.0050-1.6997 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.0500-5.9934 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-CV.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-M.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-WD.fits


In [2]:
mpl.rc('font', size=14)
mpl.rc('figure', max_open_warning = 0)

## Load TNS Data

Read a CSV file of TNS alerts already downloaded.

In [3]:
tns_all = Table.read('tns_all_thru_202206.csv', format='csv')

# Convert discovery date to MJD.
mjd = Time(tns_all['Discovery Date (UT)'], format='iso', scale='utc').mjd
tns_all['Discovery Date (MJD)'] = mjd

# Convert coordinates to decimal degree format. Do not insert into the TNS table.
coords = SkyCoord(tns_all['RA'], tns_all['DEC'], frame='icrs', unit=['hour', 'degree'])

In [4]:
tns_all

col0,ID,Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Reporting Group/s,Discovery Data Source/s,Classifying Group/s,Associated Group/s,Disc. Internal Name,Disc. Instrument/s,Class. Instrument/s,TNS AT,Public,End Prop. Period,Discovery Mag/Flux,Discovery Filter,Discovery Date (UT),Sender,Remarks,Discovery Bibcode,Classification Bibcodes,Ext. catalog/s,Discovery Date (MJD)
int64,int64,str11,str12,str12,str17,float64,str32,float64,str58,str51,str39,str51,str29,str147,str67,int64,int64,str10,float64,str13,str23,str20,str134,str19,str166,int64,float64
0,110665,AT 2022muf,01:54:29.565,+39:30:13.02,--,--,WISEA J015429.55+393013.7,--,ALeRCE,ZTF,--,ALeRCE,ZTF22aaogfil,P48 - ZTF-Cam,--,1,1,--,19.9201,r-ZTF,2022-06-08 11:22:16.000,ALeRCE,--,--,--,--,59738.4737962963
1,110664,AT 2022mue,15:54:23.380,+20:18:08.57,--,--,--,--,YSE,Pan-STARRS,--,Pan-STARRS; YSE,PS22flb,PS2 - GPC2,--,1,1,--,20.84,r-P1,2022-06-06 07:39:25.056,YSE_Bot1,--,--,--,--,59736.31904
2,110663,AT 2022mud,12:57:35.872,+09:01:54.38,--,--,SDSS J125736.49+090205.4,0.104746,SGLF,ZTF,--,SGLF,ZTF22aanzotb,P48 - ZTF-Cam,--,1,1,--,19.635,g-ZTF,2022-06-11 04:35:15.000,Perez-Fournon,--,--,--,--,59741.191145833334
3,110662,AT 2022muc,19:52:34.240,+16:11:30.70,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22clw,Gaia - Gaia-photometric,--,1,1,--,16.75,G-Gaia,2022-06-10 16:49:26.400,Gaia_Bot1,--,--,--,--,59740.701
4,110661,AT 2022mub,05:36:30.950,-70:01:58.51,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22clv,Gaia - Gaia-photometric,--,1,1,--,16.65,G-Gaia,2022-06-10 23:00:57.600,Gaia_Bot1,--,--,--,--,59740.959
5,110660,AT 2022mua,05:28:09.370,-69:09:34.96,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22clu,Gaia - Gaia-photometric,--,1,1,--,14.4,G-Gaia,2022-06-10 17:00:57.600,Gaia_Bot1,--,--,--,--,59740.709
6,110659,AT 2022mtz,05:24:40.640,-70:03:44.60,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22clt,Gaia - Gaia-photometric,--,1,1,--,15.34,G-Gaia,2022-06-10 16:59:31.200,Gaia_Bot1,--,--,--,--,59740.708
7,110658,AT 2022mty,19:43:59.340,+18:30:23.33,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22cls,Gaia - Gaia-photometric,--,1,1,--,18.22,G-Gaia,2022-06-10 04:45:07.200,Gaia_Bot1,--,--,--,--,59740.198
8,110657,AT 2022mtx,19:51:20.280,+12:41:38.58,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22clr,Gaia - Gaia-photometric,--,1,1,--,18.89,G-Gaia,2022-06-10 09:05:45.600,Gaia_Bot1,--,--,--,--,59740.379


## Perform Coordinate Match between DESI Observations and TNS

Match TNS observations within 1" of a DESI fiber. Use the `expfibermap` databases, which use an estimate of the actual fiber position (as opposed to the `TARGET_RA` and `TARGET_DEC` of the fibers).

In [5]:
redux = 'daily'

try:
    db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi')
    cursor = db.cursor()
    # cursor.execute('SET search_path TO da, public;')
    
    # Accumulate data in this table.
    desi_targets = None
    
    # Loop over all TNS alerts and perform a coordinate match with DESI observations.
    for i, (alert, coord) in enumerate(zip(tns_all, coords)):
        ra, dec = coord.ra.to_value('deg'), coord.dec.to_value('deg')
        
        query = 'SELECT f.targetid,f.fiber_ra,f.fiber_dec,c.tileid,c.night,f.mjd,r.z,r.zerr,r.zwarn,r.deltachi2,c.filename\n' \
                f'FROM {redux}.tiles_expfibermap f\n' \
                f'INNER JOIN {redux}.cumulative_tiles c ON f.cumultile_id=c.id\n' \
                f'INNER JOIN {redux}.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid\n' \
                f'WHERE q3c_radial_query( f.fiber_ra, f.fiber_dec, {ra}, {dec}, 1./3600. );'
        
        cursor.execute(query)
        rows = cursor.fetchall()
        
        if rows:
            # Convert postgresql row output to an astropy Table.
            data = Table(list(map(list, zip(*rows))),
                         names=['TARGETID', 'FIBER_RA', 'FIBER_DEC', 'TILEID', 'NIGHT', 'MJD', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FILENAME'])
            
            # hstack the postgresql rows with the TNS alert.
            # The following vstack loop ensures every row gets a match with the alert.
            tns_data = alert
            if len(data) > 1:
                for j in range(1, len(data)):
                    tns_data = vstack([tns_data, alert])
            data = hstack([data, tns_data])
            
            # Accumulate matched targets.
            if desi_targets is None:
                desi_targets = data
            else:
                desi_targets = vstack([desi_targets, data], join_type='outer')
            
        if (i+1) % 250 == 0:
            print(f'Completed {i+1} out of {len(tns_all)} alerts')
            
    # if desi_targets is not None:
    #     desi_targets['FIBER_RA'].unit = 'degree'
    #     desi_targets['FIBER_DEC'].unit = 'degree'
    
except (Exception, psycopg2.Error) as error:
    print(error)
finally:
    if db is not None:
        db.close()

Completed 250 out of 36560 alerts
Completed 500 out of 36560 alerts
Completed 750 out of 36560 alerts
Completed 1000 out of 36560 alerts
Completed 1250 out of 36560 alerts
Completed 1500 out of 36560 alerts
Completed 1750 out of 36560 alerts
Completed 2000 out of 36560 alerts
Completed 2250 out of 36560 alerts
Completed 2500 out of 36560 alerts
Completed 2750 out of 36560 alerts
Completed 3000 out of 36560 alerts
Completed 3250 out of 36560 alerts
Completed 3500 out of 36560 alerts
Completed 3750 out of 36560 alerts
Completed 4000 out of 36560 alerts
Completed 4250 out of 36560 alerts
Completed 4500 out of 36560 alerts
Completed 4750 out of 36560 alerts
Completed 5000 out of 36560 alerts
Completed 5250 out of 36560 alerts
Completed 5500 out of 36560 alerts
Completed 5750 out of 36560 alerts
Completed 6000 out of 36560 alerts
Completed 6250 out of 36560 alerts
Completed 6500 out of 36560 alerts
Completed 6750 out of 36560 alerts
Completed 7000 out of 36560 alerts
Completed 7250 out of 3

In [6]:
desi_targets

TARGETID,FIBER_RA,FIBER_DEC,TILEID,NIGHT,MJD,Z,ZERR,ZWARN,DELTACHI2,FILENAME,col0,ID,Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Reporting Group/s,Discovery Data Source/s,Classifying Group/s,Associated Group/s,Disc. Internal Name,Disc. Instrument/s,Class. Instrument/s,TNS AT,Public,End Prop. Period,Discovery Mag/Flux,Discovery Filter,Discovery Date (UT),Sender,Remarks,Discovery Bibcode,Classification Bibcodes,Ext. catalog/s,Discovery Date (MJD)
int64,float64,float64,int64,int64,float64,float64,float64,int64,float64,str71,int64,int64,str11,str12,str12,str17,float64,str32,float64,str58,str51,str39,str51,str29,str147,str67,int64,int64,str10,float64,str13,str23,str20,str134,str19,str166,int64,float64
39628522947347397,230.347315965993,31.9310854271636,20239,20220218,59629.53376982,0.230183063458812,8.13670330936725e-06,0,704.982852339745,daily/tiles/cumulative/20239/20220218/redrock-6-20239-thru20220218.fits,47,110617,AT 2022msk,15:21:23.369,+31:55:52.29,--,--,--,--,Pan-STARRS,Pan-STARRS,--,Pan-STARRS,PS22fku,PS2 - GPC2,--,1,1,--,20.73,w-P1,2022-06-07 06:28:05.664,PS2_Bot1,--,2022TNSTR1650....1C,--,--,59737.26951
39628262648845377,241.572609377006,19.9240363985427,20247,20210705,59401.20643336,0.168250956600897,4.01990663670289e-05,0,658.203603351489,daily/tiles/cumulative/20247/20210705/zbest-0-20247-thru20210705.fits,105,110558,AT 2022mqe,16:06:17.364,+19:55:26.57,--,--,--,--,Pan-STARRS,Pan-STARRS,--,Pan-STARRS,PS22fiq,PS2 - GPC2,--,1,1,--,20.55,w-P1,2022-05-30 11:55:03.648,PS2_Bot1,--,2022TNSTR1650....1C,--,--,59729.49657
39628130159174650,265.176185250919,14.2923922116857,21997,20220429,59699.48066723,0.161622051219002,3.08714486711119e-05,0,228.240458453074,daily/tiles/cumulative/21997/20220429/redrock-0-21997-thru20220429.fits,128,110535,AT 2022mph,17:40:42.278,+14:17:31.74,--,--,--,--,Pan-STARRS,Pan-STARRS,--,Pan-STARRS,PS22fhs,PS2 - GPC2,--,1,1,--,20.79,w-P1,2022-06-01 09:49:46.848,PS2_Bot1,--,2022TNSTR1650....1C,--,--,59731.40957
39627763748966538,189.734074290681,-1.05913552456076,8383,20220306,59645.36469216,0.20835137082616,1.28384459398157e-05,0,13879.3184538484,daily/tiles/cumulative/8383/20220306/redrock-7-8383-thru20220306.fits,178,110482,AT 2022mnj,12:38:56.182,-01:03:32.82,--,--,--,--,ANTARES,ZTF,--,ANTARES,ZTF22aaftzcn,P48 - ZTF-Cam,--,1,1,--,18.4679,g-ZTF,2022-06-04 04:23:43.296,ANTARES_bot,--,2022TNSTR1633....1A,--,--,59734.18314
39627763748966538,189.73403709154,-1.05913777122085,6274,20220324,59663.32227808,0.208337251131304,1.27630229481803e-05,0,11600.6685008705,daily/tiles/cumulative/6274/20220324/redrock-5-6274-thru20220324.fits,178,110482,AT 2022mnj,12:38:56.182,-01:03:32.82,--,--,--,--,ANTARES,ZTF,--,ANTARES,ZTF22aaftzcn,P48 - ZTF-Cam,--,1,1,--,18.4679,g-ZTF,2022-06-04 04:23:43.296,ANTARES_bot,--,2022TNSTR1633....1A,--,--,59734.18314
39627681850986125,345.669004196735,-4.40092267348929,25236,20211021,59509.18687241,0.0950312352466615,3.15907234344072e-06,0,3192.18104553223,daily/tiles/cumulative/25236/20211021/redrock-3-25236-thru20211021.fits,186,110474,AT 2022mnb,23:02:40.561,-04:24:03.93,--,--,SDSS J230240.57-042403.3,--,ALeRCE,ZTF,--,ALeRCE,ZTF22aanvqbv,P48 - ZTF-Cam,--,1,1,--,20.6576,g-ZTF,2022-06-07 10:20:07.002,ALeRCE,--,2022TNSTR1618....1M,--,--,59737.43063659722
39627633268360807,319.075201527494,-6.39819710400395,25547,20211021,59509.08425356,0.222653735679976,8.03764212770075e-06,0,1627.98154354095,daily/tiles/cumulative/25547/20211021/redrock-7-25547-thru20211021.fits,243,110417,AT 2022mkw,21:16:18.080,-06:23:53.97,--,--,--,--,Pan-STARRS,Pan-STARRS,--,Pan-STARRS,PS22feb,PS2 - GPC2,--,1,1,--,20.89,w-P1,2022-06-04 12:52:14.592,PS2_Bot1,--,2022TNSTR1611....1C,--,--,59734.53628
39627633268360807,319.075201527494,-6.39819710400395,25547,20211021,59509.09638793,0.222653735679976,8.03764212770075e-06,0,1627.98154354095,daily/tiles/cumulative/25547/20211021/redrock-7-25547-thru20211021.fits,243,110417,AT 2022mkw,21:16:18.080,-06:23:53.97,--,--,--,--,Pan-STARRS,Pan-STARRS,--,Pan-STARRS,PS

### Dump Output to a File

In [7]:
desi_targets.write('desi_tns_matches_thru_202206.fits', overwrite=True)

## Select Observations within a $\pm$3 Week Window around the TNS Discovery Date

In [8]:
select = np.abs(desi_targets['MJD'] - desi_targets['Discovery Date (MJD)']) < 21

In [9]:
desi_targets[select]

TARGETID,FIBER_RA,FIBER_DEC,TILEID,NIGHT,MJD,Z,ZERR,ZWARN,DELTACHI2,FILENAME,col0,ID,Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Reporting Group/s,Discovery Data Source/s,Classifying Group/s,Associated Group/s,Disc. Internal Name,Disc. Instrument/s,Class. Instrument/s,TNS AT,Public,End Prop. Period,Discovery Mag/Flux,Discovery Filter,Discovery Date (UT),Sender,Remarks,Discovery Bibcode,Classification Bibcodes,Ext. catalog/s,Discovery Date (MJD)
int64,float64,float64,int64,int64,float64,float64,float64,int64,float64,str71,int64,int64,str11,str12,str12,str17,float64,str32,float64,str58,str51,str39,str51,str29,str147,str67,int64,int64,str10,float64,str13,str23,str20,str134,str19,str166,int64,float64
39633183557290010,211.401571479868,46.0352918708225,24307,20220512,59712.2399852,0.151717815377948,1.45188747008652e-05,0,342.536314070225,daily/tiles/cumulative/24307/20220512/redrock-2-24307-thru20220512.fits,424,110233,AT 2022mdx,14:05:36.322,+46:02:07.24,--,--,SDSS J140536.39+460207.0,--,ALeRCE,ZTF,--,ALeRCE,ZTF22aandihp,P48 - ZTF-Cam,--,1,1,--,20.7592,g-ZTF,2022-06-02 06:41:05.001,ALeRCE,--,2022TNSTR1568....1M,--,--,59732.278530104166
39628385227378740,236.873131811834,25.4495715939117,7504,20220605,59736.29982708,0.581114725102594,1.06952559428867e-05,0,25808.99387604,daily/tiles/cumulative/7504/20220605/redrock-4-7504-thru20220605.fits,474,110183,AT 2022mbz,15:47:29.560,+25:26:58.34,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22cfo,Gaia - Gaia-photometric,--,1,1,--,18.08,G-Gaia,2022-05-22 21:46:04.800,Gaia_Bot1,--,2022TNSTR1559....1H,--,--,59721.907
39628497592781687,263.173099681455,30.636962630219,25610,20220516,59716.41518338,-7.97770424580597e-05,1.32295044959574e-06,0,125234.549368564,daily/tiles/cumulative/25610/20220516/redrock-3-25610-thru20220516.fits,14,110143,AT 2022mal,17:32:41.540,+30:38:12.84,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22cfk,Gaia - Gaia-photometric,--,1,1,--,17.53,G-Gaia,2022-05-22 02:21:07.200,Gaia_Bot1,--,2022TNSTR1559....1H,--,--,59721.098
2305843021716920599,199.517626297325,66.1944509569795,40123,20220510,59710.21578847,0.0776120629553853,2.86669563553839e-05,0,2791.68721711449,daily/tiles/cumulative/40123/20220510/redrock-5-40123-thru20220510.fits,17,110139,SN 2022mai,13:18:04.296,+66:11:40.06,SN Ia,0.07749,--,--,"ZTF, ATLAS","ZTF, ATLAS",ZTF,ATLAS; ZTF,ZTF22aalzyzl,"ATLAS-HKO - ATLAS-02, ATLAS-MLO - ATLAS-01, P48 - ZTF-Cam",P60 - SEDM,1,1,--,20.42,g-ZTF,2022-05-30 09:48:57.600,ZTF_Bot1,--,2022TNSTR1563....1F,2022TNSCR1641....1S,--,59729.409
39632966669830273,235.672526912134,34.3691312722889,5944,20220528,59728.30776534,2.05197222911073,0.000119854491934782,2,1150.01232803613,daily/tiles/cumulative/5944/20220528/redrock-4-5944-thru20220528.fits,223,109931,AT 2022lsk,15:42:41.410,+34:22:08.80,--,--,--,--,GaiaAlerts,GaiaAlerts,--,GaiaAlerts,Gaia22ceb,Gaia - Gaia-photometric,--,1,1,--,18.44,G-Gaia,2022-05-18 19:56:38.400,Gaia_Bot1,--,2022TNSTR1537....1H,--,--,59717.831
39627993487770793,239.388067912658,8.51192640742695,24914,20220316,59655.48832933,0.05,1.76850202395271e-05,1056,109062.424378578,daily/tiles/cumulative/24914/20220316/redrock-8-24914-thru20220316.fits,435,109718,AT 2022lkg,15:57:33.171,+08:30:42.96,--,--,--,--,ZTF,ZTF,--,ZTF,ZTF22aagpzva,P48 - ZTF-Cam,--,1,1,--,18.9781,r-ZTF,2022-04-04 08:58:12.000,ZTF_CLU_PUBLIC,--,2022TNSTR1499....1D,--,--,59673.37375
39627993487770792,239.38807069105,8.51195302140565,23352,20220413,59683.41815283,0.05,1.94601282840151e-05,1056,80983.527652743,daily/tiles/cumulative/23352/20220413/redrock-3-23352-thru20220413.fits,435,109718,AT 2022lkg,15:57:33.171,+08:30:42.96,--,--,--,--,ZTF,ZTF,--,ZTF,ZTF22aagpzva,P48 - ZTF-Cam,--,1,1,--,18.9781,r-ZTF,2022-04-04 08:58:12.000,ZTF_CLU_PUBLIC,--,2022TNSTR1499....1D,--,--,59673.37375
43977624695162029,188.384191207986,7.09298575549743,42390,20220601,59732.13811398,0.116097205774152,1.49898248876406e-05,0,517.421557724476,daily/tiles/cumulative/42390/20220601/redrock-9-42390-thru20220